# Rain Hyperparameter Initialization

The purpose of this notebook is to examine the hyperparameter associated with rain in the FMDA RNN model. The current architecture is to have an additive effect of rain on fuel moisture in the initiated model. There is a hyperparameter associated with this effect that we wish to set from a physics-based model.

## Background: Physics Model from Vanderkamp 2014

Vanderkamp (2014) establish the following physics-based model of the affect of rain on fuels. The depth of a stick is modeled, and there is assumed to be radial symmetry and symmetry along the length of the stick. During rain, the outer layer of the stick absorbs the rain. The amount of absorbed rain $P_{abs}$, in units of $kg/s$, is calculated from the diamter of the stick $2r$, length $l$ and the incident precipitation rate $P_{inc}$ (in units of $kg s^{-1}m^{-2}$):

$P_{abs}=2r l P_{inc}$

Additionally, the moisture is limited to a max value $m_{max}$, which is "determined through model calibration". For 10-h fuels, they arrived on $m_{max}=76.8\%$. This value was valibrated at a single study location.

## Translate Vanderkamp Model to our context

### Calculate Expected Change from 1mm Rain

In order to apply this physical principle to the physics-initiated RNN, we use this formula to calculate the expected increase in fuel moisture content from 1mm of rain over an hour. Additionally, we are working with 10h fuels, which correspond to roughly 1/2 inch diamter sticks, or $0.0127 m$

1mm of rain spread over $1m^2$ over an hour corresponds to 1kg of water per hour, so let $P_{inc}=1kg hr^{-1}=1/3600 \; kg s^{-1}$.
* 1mm is .001m, and if spread over $1m^2$, we have a 3-d volume of water .001 $m^3$
* The density of water is approximately 1,000kg per $m^3$, so $1,000 kgm^{-3}\cdot .001 m^3=1kg$

The percent change in fuel moisture content per hour, in units of $hr{-1}$ is calculated assuming an initial dry weight of $M$ kg. Denote this term $K$:

$$
K = \frac{\text{Weight of water per hour}\;kg\;hr^{-1}}{M\; kg},\quad\text{Units: }hr^{-1}
$$
$$
= \frac{2lr P_{inc}\; kg\;s^{-1} \cdot 3600 s\;hr{-1}}{M \; kg}
$$

The mass of the stick is $M=\text{Volume}\cdot \text{Density}$. The volume can be calculated geometrically, and the Density comes from literature. [Research suggests](https://www.engineeringtoolbox.com/wood-density-d_40.html) that the density of woods varies from 300 to 900 $kg\;m{-3}$. 

$$
K = \frac{2lr}{\pi r^2 l \rho}
$$
$$
= \frac{2}{\pi r \rho}=\frac{4}{\pi d \rho}
$$

Here, $d$ is the diameter of the stick, $0.0127 m$. So substituting this into the equation, with wood densities of 300 to 900 $kg \; m^{-3}$, we get:

$$
K = 0.11 \text{ to } 0.33
$$

In percentage terms this is $11$ to $33$ percent fuel moisture per mm of rain in an hour. As a feasibility check, working with the denser wood estimate a stick would have moisture content of 100 if exposed to 9mm of rain in an hour. [USGS](https://water.usgs.gov/edu/activity-howmuchrain-metric.html#:~:text=Slight%20rain%3A%20Less%20than%200.5,than%208%20mm%20per%20hour.) classifies this as very heavy rain, so it is feasible that a dry stick exposed to very heavy rain for an hour would absorb a lot of water.

## Translating the Vanderkamp Model

### RNN Model of Rain

The core of the physics-initiated model is an ODE built off of Mandel 2014. In the current version of the RNN (as of Jan 1 2024), the initiated model adds the effect of rain to the ODE moisture model. Call the ODE component of the model $f(m_t)$. So the modeled moisture at time $t=1$ would be the sum of the moisture model calculated on the previous timestep ($f(m_0)$) and a constant effect based on the rainfall. Let the following definitions for the model be:

* $S_{fm}$: scale parameter associated with fuel moisture. Currently (Jan 1 2014), this is calculated as the maximum observed fuel moisture value in the training data. Moving forward, this will be a hyperparameter.
* $S_{rain}$: scale parameter associated with rain. As before, this is calculated as from the training data as the maximum observed rain (units of mm).
* $R_t$: the observed rainfall data at time $t$ (units of mm)
* $K$: the hyperparameter associated with rain. It is the percent increase in fuel moisture per mm of rain per hour, in units of percentage of fuel moisture per hour millimeter $\frac{\%}{h\cdot mm}$ (??? Get Jan to Confirm)

Thus, the modeled moisture at time $t=1$ is:

$$
m_1 = f(m_0)+ K \cdot \frac{S_{fm}}{S_{rain}} \cdot R_1
$$

In this notebook we will attempt to translate the Vanderkamp model of rain into the physics-initiated RNN to get a good value of the parameter $K$. Following that, we could conduct a grid-search on that parameter around this physics-based starting value.

### Setting Parameter K

First, we will be working with 10h fuels, which corresponds to 1/2-inch sticks, per [USDA](https://www.fs.usda.gov/t-d/pubs/htmlpubs/htm05512347/index.htm#:~:text=Dead%20fuels%20are%20divided%20into,to%20absorb%20or%20lose%20moisture.), which is equal to 12.7 mm. So the numerator in the Vanderkamp model would be .0127 m. The denominator is the incoming rain $P_{inc}$, in units of $kg s^{-1} m^{-2}$. 

The physics-initiated RNN formulation will constrain the rainfall to be 1-mm over an hour. If 1 mm rain is spread over 1 $m^2$, that is 1/1000 $m^3$, equal to 1L of water. The resulting mass of 1L water would be approximately 1 kg (.001 $m^3$ times 1,000 kg/$m^3$). So, 1mm of rain in an hour corresponds to 1/3600 $kg \cdot s^{-1}m^{-2}$.

So, the absorbed water in terms of $kgs^{-1}$ would be:
$$
P_{abs}=\frac{.0127}{1/3600}=45.72
$$

**...this doesn't make sense. Why should the absorbed kg go down with increased rain, and up with less rain? What am I doing wrong?**

## Setup

In [ ]:
import sys
sys.path.append("..")

In [ ]:
# Setup
import reproducibility

# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import tensorflow as tf
import pickle, os

from data_funcs import load_and_fix_data, plot_data
from moisture_rnn import create_rnn_data, train_rnn, rnn_predict

In [ ]:
# Data
# Change directory for data read/write

dict_file='../data/raws_CO_202306.pickle' # input path of FMDA dictionaries
reproducibility_file='../data/reproducibility_dict.pickle'

# read test datasets
test_dict={}
test_dict.update(load_and_fix_data(dict_file))
print(test_dict.keys())

repro_dict={}
repro_dict.update(load_and_fix_data(reproducibility_file))
print(repro_dict.keys())
# Build Case Data
id = "CPTC2_202306010000"
case_data=test_dict[id]
case_data["hours"]=len(case_data['fm'])
case_data["h2"]=int(24*20)

## References

* Vanderkamp 2014
* Mandel 2014
* USDA Stick Diam: https://www.fs.usda.gov/t-d/pubs/htmlpubs/htm05512347/
* USGS Rain: https://water.usgs.gov/edu/activity-howmuchrain-metric.html